<h1>FFCSA demo examples</h1>

Load packages

In [1]:
LoadPackage("fsr");

misc.gd OK,	outputs.gd OK,	fsr.gd OK,	lfsr.gd OK,	nlfsr.gd OK,	filfun.gd OK,	o\
utfsr.gd OK,	drawlfsr.gd OK,	drawnlfsr.gd OK,	init.g done!!!
misc.gi OK,	outputs.gi OK,	fsr.gi OK,	lfsr.gi OK,	nlfsr.gi OK,	fsrfil.gi OK,	o\
utfsr.gi OK,	drawlfsr.gi OK,	drawnlfsr.gi OK,	read.g done!!!

You can now use 200 variables x_0 ...x_199


true

In [2]:
LoadPackage("ffcsa");

misc.gd OK,	weight.gd OK,	matrix.gd OK,	nb.gd OK,	findElm.gd OK,	findPoly.gd O\
K,	ffbases.gd  OK,	findElm.gd OK,	findPolySpecial.gd OK,	algs.gd OK,	algsmatri\
x.gd OK,	init.g done!!!
misc.gi OK,	weight.gi OK,	matrix.gi OK,	nb.gi OK,	findElm.gi OK,	findPoly.gi O\
K,	ffbases.gi  OK,	findElmSpecial.gi OK,	findPolySpecial.gi OK,	
variables
[ "a_0", "a_1", "a_2", "a_3", "a_4", "a_5", "a_6", "a_7", "a_8", "a_9" ]
[ "b_0", "b_1", "b_2", "b_3", "b_4", "b_5", "b_6", "b_7", "b_8", "b_9" ]
[ "d_0", "d_1", "d_2", "d_3", "d_4", "d_5", "d_6", "d_7", "d_8", "d_9", 
  "d_10", "d_11", "d_12", "d_13", "d_14", "d_15", "d_16", "d_17", "d_18" ]
algs.gi OK,	algsmatrix.gi OK,	read.g done!!!


true

 <h2>Multiplication expressions for ${\mathbb F}_{2^4}$</h2>
 

We use irreducible polynomial $f(x)=x^4+x+1$ with root $\rho\in{\mathbb F}_{2^4}$ and the polynomial basis 
${\rm PB}=\{1, \rho, \rho^2, \rho^3\}$. 
The GAP code below shows the setup


In [7]:
K := GF(2);; x := X(K, "x");;
f := x^4+x+1;; F := FieldExtension(K,  f);;
PB := GeneratePB(F, RootOfDefiningPolynomial(F));


Basis( GF(2^4), [ Z(2)^0, Z(2^4), Z(2^4)^2, Z(2^4)^3 ] )

The `ChooseFieldElms($F$)` method prepares vectors $avec=\left[a_0, \dots , a_{m-1}\right]$ and 
$bvec=\left[b_0,\dots,b_{m-1}\right]$ with default direction `To`, where $m=[{\mathbb F}_{2^4}:{\mathbb F}_{2}]$ is the degree of extension.
Note that $a_i, b_j$ are not coefficients, but 
[GAP variables](https://docs.gap-system.org/doc/ref/chap4_mj.html#X7A4C2D0E7E286B4F) 
to allow symbolic computation. 
 

In [8]:
ChooseFieldElms(F);


variables
[ "a_0", "a_1", "a_2", "a_3" ]
[ "b_0", "b_1", "b_2", "b_3" ]
[ "d_0", "d_1", "d_2", "d_3", "d_4", "d_5", "d_6" ]


Next we generate the multiplication matrix ${\rm U}$ from basis ${\rm PB}$ and variables in $avec$

In [10]:
U := MatrixUExpression(PB, avec);
for i in U do Display(i); od;

[ a_0, a_3, a_2, a_1 ]
[ a_1, a_0+a_3, a_2+a_3, a_1+a_2 ]
[ a_2, a_1, a_0+a_3, a_2+a_3 ]
[ a_3, a_2, a_1, a_0+a_3 ]


[ [ a_0, a_3, a_2, a_1 ], [ a_1, a_0+a_3, a_2+a_3, a_1+a_2 ],   [ a_2, a_1, a_0+a_3, a_2+a_3 ], [ a_3, a_2, a_1, a_0+a_3 ] ]

Method `FFA_mult_matrixU` produces the expressions used for the hardware implementation
by multiplying ${\rm U}$ and $bvec$
 

In [12]:
mult := FFA_mult_matrixU(PB, avec, bvec);; 
for i in mult do Display(i); od; 

a_0*b_0+a_1*b_3+a_2*b_2+a_3*b_1
a_0*b_1+a_1*b_0+a_1*b_3+a_2*b_2+a_2*b_3+a_3*b_1+a_3*b_2
a_0*b_2+a_1*b_1+a_2*b_0+a_2*b_3+a_3*b_2+a_3*b_3
a_0*b_3+a_1*b_2+a_2*b_1+a_3*b_0+a_3*b_3


For example, to drive the multiplier output $c_0$, 
the expression $a_0b_0+a_1b_3+a_2b_2+a_3b_1$ 
must be implemented in hardware. 